# Instalación de dependencias para librerías y bases de datos

In [1]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!git clone https://github.com/UN-GCPDS/eeg_models.git
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.

  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-tfjzuhg0
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-tfjzuhg0
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit d174df9958b6638156dcfe03996a6307e631a6a2
  Preparing metadata (setup.py) ... done
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=32972807 sha256=30fc89ef0434bc16b1e41063dea2d16942c73f122d1995c15539df81ef0cb8c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-jx33ty85/wheels/ae/48/8d/edf617d5fe8f03b17aa26306a04abdfcc605b218d8e6deac83
Successfully built gcpds-databases
Cloning into 'eeg_models'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 134 (delta 81), reused 72 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (134/134), 79.89 KiB | 2.

# Importación de librerías

In [2]:
# load databases
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import sys
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
sys.path.append('/kaggle/working/eeg_models')

from utils import train
import numpy as np

2025-04-25 12:45:06.898323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745585107.188220      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745585107.282476      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Creación del modelo

In [3]:
from models import PST_attention


# Definición de las dimensiones de entrada
C = 64  # Número de canales
T = 320  # Número de muestras por segmento

# Parámetros específicos del modelo
model_params = {
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'last_layer': 'Dense'
}

# Argumentos generales del modelo
model_args = {
    'model_name': 'PST_attention',
    'nb_classes': 2,
    'autoencoder': False
}

# Inicialización del modelo con los parámetros definidos
model = PST_attention(**model_params, Chans=C, Samples=T)

model.summary()

I0000 00:00:1745585120.409184      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745585120.410016      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 320, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None, 64, 320, 3)          │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_1 (Conv2D)                    │ (None, 64, 296, 40)         │           3,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_2 (Conv2D)                    │ (None, 1, 296, 40)          │         102,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1, 296, 40)          │             160 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 1, 296, 40)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 15, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 15, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 15, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ F_1 (Flatten)                        │ (None, 600)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │           1,202 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_activation (Activation)          │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 106,866 (417.45 KB)

 Trainable params: 106,786 (417.13 KB)

 Non-trainable params: 80 (320.00 B)

# Pruebas con GIGA

In [4]:
# Nombre de la base de datos y carga
db_name = 'GIGA_MI_ME'
db_path = '/kaggle/input/giga-science-gcpds/GIGA_MI_ME'
db = GIGA_MI_ME(db_path)

# Frecuencia de muestreo
fs = db.metadata['sampling_rate']

# Nombres de los canales EEG
eeg_ch_names = [
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

# Definición de los bancos de frecuencia y ventana temporal
f_bank = np.array([[4., 40.]])        # Banda de frecuencia de interés
vwt = np.array([[2.5, 5]])            # Ventana temporal para la representación TF

# Representación tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# Argumentos para la función de carga de datos
load_args = {
    'db': db,
    'eeg_ch_names': eeg_ch_names,
    'fs': fs,
    'f_bank': f_bank,
    'vwt': vwt,
    'new_fs': 128.0,
    'tf_repr': tf_repr
}

# Selección de sujetos (excluyendo sujetos 29 y 34)
subjects = np.arange(1, db.metadata['subjects'] + 1)
subjects = np.delete(subjects, [28, 33])

In [5]:
# === Imports ===
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

# === Parámetros generales ===
seed = 23
verbose = 0

# === Callbacks de entrenamiento ===
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=30,
    verbose=verbose,
    mode='min',
    min_delta=0.01,
    min_lr=0
)

terminate_on_nan = TerminateOnNaN()

callbacks = [reduce_lr_on_plateau, terminate_on_nan]

# === Configuración de validación cruzada ===
cv_args = {
    'cv': StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
}

# === Argumentos de compilación del modelo ===
compile_args = {
    'loss': SparseCategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 1e-2
}

# === Argumentos del ajuste (fit) del modelo ===
fit_args = {
    'epochs': 500,
    'verbose': verbose,
    'callbacks': callbacks
}

In [6]:
all_results = []

for sbj in subjects[:]: 
    print('sbj = ', sbj)
    load_args['sbj'] = sbj
    results = train(model, db_name, tf_repr, load_args, cv_args, model_args, compile_args, fit_args, seed)
    all_results.append((sbj, results))

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.


I0000 00:00:1745585132.615927     121 service.cc:148] XLA service 0x7e97a0113d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745585132.616978     121 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745585132.616998     121 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745585133.029434     121 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-25 12:45:36.562556: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[160,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[160,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 993ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
sbj =  2
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
sbj =  3
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
sbj =  4
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
sbj =  5
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 13:12:58.331281: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[159,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 13:12:58.566223: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.235056253s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[159,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step
sbj =  6
Resampling from 512.000000 to 128.000000 Hz.


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 566ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 566ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 570ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
sbj =  47
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 17:48:19.355586: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[153,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[153,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 17:48:19.548499: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.192996483s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[153,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[153,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
sbj =  48
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 589ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
sbj =  49
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
sbj =  50
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
sbj =  51
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 18:16:43.500589: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[155,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[155,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 18:16:43.689109: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.188710297s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[155,40,64,296]{3,2,1,0}, u8[0]{0}) custom-call(f32[155,40,1,296]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
sbj =  52
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step


In [7]:
import pickle

with open(f'results_{model_args["model_name"]}.pkl', 'wb') as f:
    pickle.dump(all_results, f)